In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
import plotly.express as px
sb.set() # set the default Seaborn style for graphics

In [ ]:
listing = pd.read_csv("listings.csv")
calendar = pd.read_csv("calendar.csv")
reviews = pd.read_csv("reviews.csv")
rating = pd.DataFrame(listing["review_scores_rating"])

In [ ]:
reqData = ['price', 'name', 'longitude', 'latitude', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_value', 'review_scores_location', 'number_of_reviews', 'host_response_time','host_response_rate']

listingLess = listing[reqData]

# Removed below 80 reviews as they are mostly outliers and also to allow for less clutter in the visualization
ldf = listingLess[(listingLess.review_scores_rating >= 80) & (listingLess.number_of_reviews >= 30)]

p = ldf['price'].replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)
hrr = ldf['host_response_rate'].replace( '[\%,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)

ldf['price'] = p
ldf['host_response_rate'] = hrr 

mapStyleOSM = "open-street-map"
mapStyleL = "light"
mapStyleD = "dark"

colors = ["royalblue","crimson","lightseagreen","orange","fuchsia"]
print(ldf.head())

In [ ]:
# This is to get a better understanding of the data we are looking at

f, axes = plt.subplots(4, 2, figsize=(35, 40))
sb.countplot(x = "review_scores_communication", data = ldf, orient = "h", ax = axes[0, 0])
sb.countplot(x = "review_scores_accuracy", data = ldf, orient = "h", ax = axes[0, 1])
sb.countplot(x = "review_scores_location", data = ldf, orient = "h", ax = axes[1, 0])
sb.countplot(x = "review_scores_rating", data = ldf, orient = "h", ax = axes[1, 1])
sb.countplot(x = "review_scores_value", data = ldf, orient = "h", ax = axes[2, 0])
sb.countplot(x = "host_response_time", data = ldf, orient = "h", ax = axes[2, 1])
sb.countplot(x = "host_response_rate", data = ldf, orient = "h", ax = axes[3, 0])
sb.countplot(x = "price", data = ldf, orient = "h", ax = axes[3, 1])



In [25]:
import plotly.graph_objects as go
# token to access mapbox API
token = "pk.eyJ1IjoicmFuZG9tYW5kbyIsImEiOiJjazhvaHZkNnYwMWozM2xwYWk1d2dsdXVhIn0.oOoOYk94pLXAbWI3KERHLQ"
cldf = ldf.copy()

# list of quarterly quantiles
Qs = [.25, .5, .75, 1]
def label4(row, **kwargs):
    if row[kwargs['col']] <= cQs[0]:
        val = kwargs['vals'][0]
    elif (cQs[1] >= row[kwargs['col']]) & (row[kwargs['col']] > cQs[0]):
        val = kwargs['vals'][1]
    elif (cQs[2] >= row[kwargs['col']]) & (row[kwargs['col']] > cQs[1]):
        val = kwargs['vals'][2]
    else:
        val = kwargs['vals'][3]
    return val

def label2(row, **kwargs):
    if row[kwargs['col']] == kwargs['quant']:
        val = kwargs['vals'][0]
    else:
        val = kwargs['vals'][1]
    return val


# Here, we are creating the tags for each listing that will be shown in the map at the end
# while this method seems quite ugly, it would be unnecessarily complicated to created a full automated solution (i.e. look thru something like a dictionary that had the response tags)
# Because some columns have almost a binary score, (e.g. location has majority of its values at 9 or 10), different functions are used (label4() and label2()). Label 4 give back 4 tags defined over 4 different ranges respectively, same goes for label2 w/ 2 tags.
cQs = []
for j in range(len(Qs)):
    cQs.append(cldf['price'].quantile(Qs[j]))
print(cQs)
cldf['feat0'] = cldf.apply(label4, col='price', vals=['very cheap', 'affordable', 'pricey', 'high-end'], axis=1)

cQs = []
for j in range(len(Qs)):
    cQs.append(cldf['review_scores_rating'].quantile(Qs[j]))
cldf['feat1'] = cldf.apply(label4, col='review_scores_rating', vals=['poor review', 'lower than average review', 'higher than average review', 'best review'], axis=1)

# below are columns that monstly have 1 category or another, hence use of label2
cldf['feat2'] = cldf.apply(label2, col='host_response_rate', vals=['very responsive', ''], quant=100, axis=1)
cldf['feat3'] = cldf.apply(label2, col='review_scores_cleanliness', vals=['very clean', ''], quant=10, axis=1)
cldf['feat4'] = cldf.apply(label2, col='review_scores_location', vals=['good location', ''], quant=10, axis=1)
cldf['feat5'] = cldf.apply(label2, col='review_scores_value', vals=['good value', ''], quant=10, axis=1)
cldf['feat6'] = cldf.apply(label2, col='review_scores_accuracy', vals=['accurate listing', ''], quant=10, axis=1)

# uncomment to see data
# pd.set_option('display.max_rows', cldf.shape[0]+1)
# print(cldf['feat0'])
# print(cldf['feat1'])
# print(cldf['feat2'])
# print(cldf['feat3'])
# print(cldf['feat4'])
# print(cldf['feat5'])
# print(cldf['feat6'])

[70.0, 95.0, 125.0, 999.0]


In [26]:
fig = go.Figure()
# define how data is groups and represented in map
limits = [(80,85),(86,90),(91,95),(96,100)]

# most of the usage of plotly is quite self-explanatory
for i in range(len(limits)):
    lim = limits[i]
    # define the data to look at for this grouping
    cat_df = cldf[(cldf.review_scores_rating >= lim[0]) & (cldf.review_scores_rating <= lim[1])]
    
    fig.add_trace(dict(
        type = "scattermapbox",
        lon = cat_df['longitude'],
        lat = cat_df['latitude'],
        # text in box when hovering over listing in map
        text = cat_df['name'] + " - $" + cat_df['price'].astype(str) + "</br>" + \
         cat_df['review_scores_rating'].astype(str) + " Review Rating",
        marker = dict(
            size = cat_df['review_scores_rating']/8,
            color = colors[i],
            opacity = .8,
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])
        )
    )
    


fig.update_layout(
    mapbox={
        "accesstoken": token, 
        "style": mapStyleL,
        "zoom": 10,
        "center":dict(
                lat=cldf['latitude'][0],
                lon=cldf['longitude'][0]
            ),
    },
    title_text = 'Airbnb review score ratings in categories',
    showlegend = True,
    geo = dict(
        scope = 'usa',
        landcolor = 'rgb(217, 217, 217)',
    ),
    height = 800,
    
)
fig.show()

In [27]:
fig = go.Figure()
limits = [8,9,10]

for i in range(len(limits)):
    lim = limits[i]
    cat_df = cldf[(cldf.review_scores_location == lim)]
    
    fig.add_trace(dict(
        type = "scattermapbox",
        lon = cat_df['longitude'],
        lat = cat_df['latitude'],
        text = cat_df['name'] + " - $" + cat_df['price'].astype(str) + "</br>" + \
         cat_df['review_scores_rating'].astype(str) + " Review Rating",
        marker = dict(
            size = np.sqrt(cat_df['price']),
            color = colors[i],
            opacity = 0.85,
        ),
        name = '{0}'.format(lim)
        )
    )
    

    

fig.update_layout(
    mapbox={
        "accesstoken": token, 
        "style": "light",
        "zoom": 10,
        "center":dict(
                lat=cldf['latitude'][0],
                lon=cldf['longitude'][0]
            ),
    },
    title_text = 'Airbnb grouped location review scores',
    showlegend = True,
    geo = dict(
        scope = 'usa',
        landcolor = 'rgb(217, 217, 217)',
    ),
    height = 800,
    
)
fig.show()

In [28]:
fig = go.Figure()
limits = [(0,50),(51,100),(101,200),(201,1000)]
# limits = [(0,65),(66,120),(121,190),(191,300),(301,600)]

for i in range(len(limits)):
    lim = limits[i]
    cat_df = cldf[(cldf.price >= lim[0]) & (cldf.price <= lim[1])]
    
    fig.add_trace(dict(
        type = "scattermapbox",
        lon = cat_df['longitude'],
        lat = cat_df['latitude'],
        text = cat_df['name'] + " - $" + cat_df['price'].astype(str) + "</br>" + \
         cat_df['review_scores_rating'].astype(str) + " Review Rating",
        marker = dict(
            size = np.sqrt(cat_df['price']),
            color = colors[i],
            opacity = .9,
        ),
        name = '${0} - {1}'.format(lim[0],lim[1])
        )
    )
    

    

fig.update_layout(
    mapbox={
        "accesstoken": token, 
        "style": mapStyleL,
        "zoom": 10,
        "center":dict(
                lat=cldf['latitude'][0],
                lon=cldf['longitude'][0]
            ),
    },
    title_text = 'Airbnb grouped listing prices',
    showlegend = True,
    geo = dict(
        scope = 'usa',
        landcolor = 'rgb(217, 217, 217)',
    ),
    height = 800,
    
)
fig.show()

In [31]:
fig = go.Figure()
# limits = [(0,50),(51,100),(101,200),(201,1000)]
# limits = [(0,65),(66,120),(121,190),(191,300),(301,600)]  

limits = [0]
for j in range(len(Qs)):
    limits.append(cldf['price'].quantile(Qs[j]))
# print(limits)

x = ['very cheap', 'affordable', 'pricey', 'high-end']
for i in range(len(x)):
    # lim = limits[i]
    cat_df = cldf[(cldf.feat0 == x[i])]
    
    fig.add_trace(dict(
        type = "scattermapbox",
        lon = cat_df['longitude'],
        lat = cat_df['latitude'],
        # here, the tags that was added above to each listing is added into the popup box when howvering over listings on the map
        text = cat_df['name'] + " - $" + cat_df['price'].astype(str) + "</br>Review Rating: " + \
        cat_df['review_scores_rating'].astype(str) + "</br>Tags:</br>" + \
        cat_df['feat0'] + "</br>" + cat_df['feat1'] + "</br>" + cat_df['feat2'] + "</br>" + cat_df['feat3'] + "</br>" +\
        cat_df['feat4'] + "</br>" + cat_df['feat5'] + "</br>" + cat_df['feat6'],
        # text = "{0} - ${1}</br>Review Rating: {2}</br>Tags:{3} | {4} | {5} | {6} |".format(cat_df['name'], cat_df['price'].astype(str), cat_df['review_scores_rating'].astype(str), cat_df['feat0'], cat_df['feat1'], cat_df['feat2'], cat_df['feat3']),
        marker = dict(
            size = 11,
            color = colors[i],
            opacity = .9,
        ),
        name = '${0} - {1}'.format(limits[i],limits[i+1])
        )
    )
    

fig.update_layout(
    mapbox={
        "accesstoken": token, 
        "style": mapStyleL,
        "zoom": 10,
        "center":dict(
                lat=cldf['latitude'][0],
                lon=cldf['longitude'][0]
            ),
    },
    title_text = 'Airbnb grouped listing prices',
    showlegend = True,
    geo = dict(
        scope = 'usa',
        landcolor = 'rgb(217, 217, 217)',
    ),
    height = 800,
    width = 1000,
    
)
fig.show()